In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeat
from scipy.ndimage import gaussian_filter
from siphon.ncss import NCSS
from netCDF4 import Dataset, num2date
from metpy.plots import StationPlot

In [2]:
def near_neighbor(xi,yi,xk,yk,phi):
    grid = np.zeros((len(yi),len(xi)))
    for e in range(len(xi)):
        for f in range(len(yi)):
            dum_dist = 10000000.
            for g in range(len(xk)):
                dist = (((xk[g]-xi[e])**2) + ((yk[g]-yi[f])**2))**(0.5)
                if (dist <= dum_dist):
                    dum_dist = dist
                    dum_phi = phi[g]
                elif (dist == dum_dist):
                    dum_phi = (dum_phi + phi[g])/2.
            grid[f,e] = dum_phi
    return gaussian_filter(grid,sigma=2)

def barnes_guess(x_i,y_i,x_k,y_k,phi_0,roi):
    grid = np.zeros((len(y_i),len(x_i)))
    # Loop through the grid and update the initial grid with the observations
    for l in range(len(x_i)):
        for m in range(len(y_i)):
            sum_weight = 0.
            num_weight = 0.
            weight     = 0.
            for p in range(len(phi_0)): # Loop for the observations
                # Determining the distance between the observation and a grid point
                rad2 = ((x_k[p]-x_i[l])**2 + (y_k[p]-y_i[m])**2)**(1/2)
                weight = np.exp(-(rad2**2)/(roi**2))
                sum_weight+=weight
                weight_phi = weight * phi_0[p] # 
                num_weight+=weight_phi
            grid[m,l] = num_weight/sum_weight
    return gaussian_filter(grid, sigma=2)

def cressman(x_i, y_i, x_k, y_k, phi_0, guess_field):
    # Estimate of grid at Obs points
    h = np.zeros(len(phi_0)).astype('float')

    # Assimilated Grid
    grid = np.zeros((len(y_i),len(x_i)))
    #print(grid.shape)

    # Begin the Cressman Scheme, doing three passes with Radius of Influence (roi)
    # of 80, 50, and 40 grid points
    for i in [1,2,3]:
        if (i == 1):
            roi = 15.**2.
        elif (i == 2):
            roi = 10.**2.
        else:
            roi = 5.**2.

            # Calculating the values of the grid at the observation points through simple linear interpolation
        for p in range(len(phi_0)):
            if (np.int(y_k[p]) in y_i) & ((np.int(x_k[p]) in x_i)):
                a = np.where(y_i == np.int(y_k[p]))[0][0]
                a2 = np.abs(y_i[a] - (y_k[p]))
                b = np.where(x_i == np.int(x_k[p]))[0][0]
                b2 = np.abs(x_i[b] - (x_k[p]))
                h[p] = (((guess_field[a,b]*(1-b2)+guess_field[a,b+1]*b2))*(1-a2) +
                        ((guess_field[a-1,b]*(1-b2)+guess_field[a-1,b+1]*b2))*(a2))
            else:
                h[p] = phi_0[p]

    # Loop through the grid and update the initial grid with the observations
        for l in range(len(x_i)):
            for m in range(len(y_i)):
                sum_weight = 0.
                num_weight = 0.
                weight     = 0.
                for p in range(len(phi_0)): # Loop for the observations
                    # Determining the distance between the observation and a grid point
                    rad2 = ((x_k[p]-x_i[l])**2 + (y_k[p]-y_i[m])**2)
                    #print(dist2)
                    if (rad2 <= roi):
                        weight = (roi - rad2)/(roi + rad2) # Setting the weight if within the roi
                    else:
                        weight = 0. # Setting the weight of the ob to zero if outside the roi
                    sum_weight+=weight
                    weight_phi = weight * (phi_0[p] - h[p]) # 
                    num_weight+=weight_phi
                if sum_weight > 0:
                    grid[m,l] = guess_field[m,l] + (num_weight/sum_weight) # Updating the grid
                else:
                    grid[m,l] = guess_field[m,l]



        guess_field = grid   # Saving the grid to use in the next iteration
    return grid

def barnes(x_i, y_i, x_k, y_k, phi_0, guess_field):
    # Estimate of grid at Obs points
    h = np.zeros(len(phi_0)).astype('float')

    # Assimilated Grid
    grid = np.zeros((len(y_i),len(x_i)))

    # Begin the Cressman Scheme, doing three passes with Radius of Influence (roi)
    # of 80, 50, and 40 grid points
    roi = 5
    for i in [1,2]:
        if (i == 1):
            gamma = 1
        elif (i == 2):
            gamma = 0.33

        # Calculating the values of the grid at the observation points through simple linear interpolation
        for p in range(len(phi_0)):
            if (np.int(y_k[p]) in y_i) & ((np.int(x_k[p]) in x_i)):
                a = np.where(y_i == np.int(y_k[p]))[0][0]
                a2 = np.abs(y_i[a] - (y_k[p]))
                b = np.where(x_i == np.int(x_k[p]))[0][0]
                b2 = np.abs(x_i[b] - (x_k[p]))
                h[p] = (((guess_field[a,b]*(1-b2)+guess_field[a,b+1]*b2))*(1-a2) +
                        ((guess_field[a-1,b]*(1-b2)+guess_field[a-1,b+1]*b2))*(a2))
            else:
                h[p] = phi_0[p]

        # Loop through the grid and update the initial grid with the observations
        for l in range(len(x_i)):
            for m in range(len(y_i)):
                sum_weight = 0.
                num_weight = 0.
                weight     = 0.
                for p in range(len(phi_0)): # Loop for the observations
                    # Determining the distance between the observation and a grid point
                    rad2 = ((x_k[p]-x_i[l])**2 + (y_k[p]-y_i[m])**2)**(1/2)
                    weight = np.exp(-(rad2**2)/(gamma*roi**2))
                    sum_weight+=weight
                    weight_phi = weight * (phi_0[p] - h[p]) # 
                    num_weight+=weight_phi
                if sum_weight > 0:
                    grid[m,l] = guess_field[m,l] + (num_weight/sum_weight) # Updating the grid
                else:
                    grid[m,l] = guess_field[m,l]



        guess_field = grid   # Saving the grid to use in the next iteration
    return grid

In [3]:
    # Data from https://ruc.noaa.gov/raobs/ mandatory levels for U.S.
    # Min Lat: 20 N
    # Max Lat: 70 N
    # Min Lon: -145 E
    # Max Lat: -50 E

    import re
    from datetime import datetime
    filename = '/Users/kgoebber/nwp/20171209_12_RAOB_data.txt'

    file_data = open(filename, 'r')

    date = []
    lat = []
    lon = []
    stid = []
    hght_500 = []

    for line in file_data.readlines():
        tmp = line.split()
        if (tmp[0] == '254'):
            _, hour, day, month, year = tmp
            date_string = '%d%s%02d%02d'%(int(year),month,int(day),int(hour))
            date.append(datetime.strptime(date_string,'%Y%b%d%H'))
        elif (tmp[0] == '1'):
            if (len(tmp) == 6):
                west_test = re.split(' | N |S |N|S|W', tmp[3])
                if (len(west_test) == 3):
                    west = -1
                else:
                    west = 1
                south_test = re.split(' | N|N|E|W', tmp[3])
                if (len(south_test) != 3):
                    south = -1
                else:
                    south = 1
                tlat, tlon = re.split(' | N |S |N|S|E|W', tmp[3])[:2]
            if (len(tmp) == 7):
                west_test = re.split('W', tmp[4])
                if (len(west_test) == 2):
                    west = -1
                else:
                    west = 1
                south_test = re.split('N', tmp[3])
                if (len(south_test) == 2):
                    south = 1
                else:
                    south = -1
                tlat = re.split('N|S', tmp[3])[0]
                tlon = re.split('E|W', tmp[4])[0]
            lat.append(south*float(tlat))
            lon.append(west*float(tlon))
        elif (tmp[0] == '3'):
            stid.append(tmp[1])
        elif (tmp[0] == '4'):
            if (float(tmp[1])/10. == 500):
                hght_500.append(float(tmp[2]))

    # Set up geographic region
    LLLon = -140.
    LLLat = 19.
    URLon = -50.
    URLat = 70

    # Set values for assimilation grid (1x1 degree lat/lon)
    lats = np.arange(URLat,LLLat-1,-1)
    lons = np.arange(LLLon,URLon+1,1)

    # Bring in Actual GFS Analysis
    actual_gfs_500 = Dataset('/Users/kgoebber/python_notebooks/gfsanl_3_20171209_1200_000.nc','r')
    actual_time = actual_gfs_500.variables['time']
    actual_vtimes = num2date(actual_time[:],units=actual_time.units)
    print('Analysis Time: {}'.format(actual_vtimes[0]))
    actual_hght_500 = actual_gfs_500.variables['Geopotential_height'][0,0,:,:]
    gfs_lats = actual_gfs_500.variables['lat'][:]
    gfs_lons = actual_gfs_500.variables['lon'][:]
    iULat = list(gfs_lats).index(URLat)
    iLLat = list(gfs_lats).index(LLLat)+1
    iULon = list(gfs_lons).index(360+URLon)+1
    iLLon = list(gfs_lons).index(360+LLLon)
    gfs_hght_500 = actual_hght_500[iULat:iLLat,iLLon:iULon]

Analysis Time: 2017-12-09 12:00:00


In [4]:
def plot(FG_type):
    clons, clats = np.meshgrid(lons,lats)
    
    if FG_type == 'gfs':
        # Get First Guess field from 12 hour forecast of GFS
        data_gfs_first_guess = Dataset('/Users/kgoebber/python_notebooks/gfs_3_20171209_0000_012.nc','r')
        FG_time = data_gfs_first_guess.variables['time']
        FG_vtimes = num2date(FG_time[:],units=FG_time.units)
        FG_hght_500 = data_gfs_first_guess.variables['Geopotential_height'][0,0,:,:]
        FG = FG_hght_500[iULat:iLLat,iLLon:iULon]
        FG_title = 'GFS Fcst'
    elif FG_type == 'NN':
        FG = near_neighbor(lons,lats,lon,lat,hght_500)
        FG_title = 'Nearest Neighbor'
    # Set firstguess field for Cressman Scheme
    # Use the Nearest Neighbor approach to initially fill the grid
    elif FG_type == 'barnes':
        FG = barnes_guess(lons,lats,lon,lat,hght_500,3)
        FG_title = 'Barnes'
    
    # Generate Analyzed Fields with Cressman and Barnes
    cress_hght_500 = cressman(lons,lats,lon,lat,hght_500,FG)
    barnes_hght_500 = barnes(lons,lats,lon,lat,hght_500,FG)

    print("First Guess Field: {}".format(FG_type))
    print("First Guess Error: {:.2f}".format(np.average(gfs_hght_500-FG)))
    print("Average Cressman Analysis Error: {:.2f}".format(np.average(gfs_hght_500-cress_hght_500)))
    print("Average Barnes Analysis Error: {:.2f}".format(np.average(gfs_hght_500-barnes_hght_500)))
    
    # Get data to plot state and province boundaries
    states_provinces = cfeat.NaturalEarthFeature(
            category='cultural',
            name='admin_1_states_provinces_lakes',
            scale='50m',
            facecolor='none')

    plotcrs = ccrs.PlateCarree()

    fig = plt.figure(1,figsize=(20,15))
    ax = fig.add_subplot(211,projection=plotcrs,label='top')

    ax.set_extent([-130, -65, 20, 50])
    ax.coastlines('50m')

    ax.add_feature(states_provinces,edgecolor='black',linewidth=0.5)
    # Set up station plotting using only every third element from arrays for plotting
    stationplot = StationPlot(ax, lon, lat, transform=ccrs.PlateCarree(), fontsize=12)
    stationplot.plot_parameter('C', hght_500)

    cs = ax.contour(clons,clats,cress_hght_500,np.arange(0,7000,60),colors='r',linestyles='dashed')
    plt.clabel(cs,fmt='%d')
    cs3 = ax.contour(clons,clats,gfs_hght_500,np.arange(0,7000,60),colors='k')
    plt.clabel(cs3,fmt='%d')

    plt.title('500-hPa Geopotential Heights (Actual Analysis - black; Cressman Analysis - red)',loc='left')
    plt.title(date[0], loc='right')

    ax2 = fig.add_subplot(212, projection=plotcrs,label='bottom')
    ax2.set_extent([-130, -65, 20, 50])
    ax2.coastlines('50m')
    ax2.add_feature(states_provinces,edgecolor='black',linewidth=0.5)
    # Set up station plotting using only every third element from arrays for plotting
    stationplot = StationPlot(ax2, lon, lat, transform=ccrs.PlateCarree(), fontsize=12)
    stationplot.plot_parameter('C', hght_500)

    cs2 = ax2.contour(clons,clats,barnes_hght_500,np.arange(0,7000,60),colors='b',linestyles='dotted')
    plt.clabel(cs2,fmt='%d')
    cs4 = ax2.contour(clons,clats,gfs_hght_500,np.arange(0,7000,60),colors='k')
    plt.clabel(cs4,fmt='%d')

    plt.title('500-hPa Geopotential Heights (Actual Analysis - black; Barnes Analysis - blue)',loc='left')
    plt.title(date[0], loc='right')

    plt.tight_layout()
    plt.show()

In [5]:
from ipywidgets import interact, widgets

interact(plot, FG_type = widgets.Dropdown(options={'GFS Fcst':'gfs','Nearest Neighbor':'NN','Barnes':'barnes'},
                                          description='First Guess: '));

interactive(children=(Dropdown(description='First Guess: ', options={'GFS Fcst': 'gfs', 'Nearest Neighbor': 'NN', 'Barnes': 'barnes'}, value='gfs'), Output()), _dom_classes=('widget-interact',))